In [1]:
import streamlit as st
import pandas as pd
import plotly.express as px
import numpy as np

In [21]:
movement = pd.read_csv('..\pitch_movement.csv')
outcomes = pd.read_csv('..\pitch-arsenal-stats.csv')
arsenal = movement.merge(outcomes,on=['pitcher_id','pitch_type','year'])
#arsenal.to_csv('arsenal.csv')
arsenal.describe()

,year,pitcher_id,avg_speed,pitches_thrown,total_pitches,pitches_per_game,pitcher_break_z,league_break_z,diff_z,rise,...,ba,slg,woba,whiff_percent,k_percent,put_away,est_ba,est_slg,est_woba,hard_hit_percent
count,3426.000000,3426.000000,3426.000000,3426.000000,3426.000000,3426.000000,3426.000000,3426.000000,3426.000000,3426.000000,...,3426.000000,3426.000000,3426.000000,3426.000000,3426.000000,3425.000000,3426.000000,3426.000000,3426.000000,3426.000000
mean,2021.499124,606482.857268,87.764361,364.442790,1398.770870,8.636102,30.156509,-30.062580,-0.100671,-0.687099,...,0.241151,0.397412,0.317387,26.413631,23.003269,18.768905,0.240226,0.392156,0.308306,36.947636
std,0.500072,63372.240992,5.874571,270.752626,753.190338,4.650295,13.117999,12.705216,3.009087,10.840025,...,0.075510,0.147740,0.087737,10.375969,11.824378,6.794912,0.058325,0.122134,0.072453,10.822396
min,2021.000000,425794.000000,65.800000,32.000000,486.000000,3.000000,8.200000,-73.500000,-14.400000,-52.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.041000,0.059000,0.071000,0.000000
25%,2021.000000,572971.000000,83.700000,171.000000,815.000000,5.030864,18.900000,-36.600000,-1.900000,-7.000000,...,0.193000,0.300000,0.261000,18.600000,14.400000,14.300000,0.202000,0.312000,0.259000,30.000000
50%,2021.000000,621244.000000,87.600000,292.000000,1086.000000,6.703704,29.100000,-29.600000,-0.100000,0.000000,...,0.239000,0.387000,0.315000,25.500000,21.750000,18.800000,0.240000,0.385000,0.307000,36.900000
75%,2022.000000,657376.000000,92.900000,479.000000,1946.250000,12.013889,37.575000,-19.600000,1.700000,6.000000,...,0.288000,0.479000,0.372000,33.100000,30.500000,22.900000,0.278000,0.464000,0.355000,44.300000
max,2022.000000,693821.000000,100.800000,1861.000000,3274.000000,20.209877,74.300000,-9.300000,10.300000,41.000000,...,0.568000,1.231000,0.750000,66.900000,84.800000,57.100000,0.534000,1.088000,0.675000,76.900000


In [11]:
#create a header for the streamlit page with an option to filter the data with two select boxes
#users can decide what year they would like to observes as well as what distinct pitch they would like data on

st.header('MLB pitch comp app')

st.write("""
##### Use velocity and movement filters to discover comparable pitches
""")

year_array = arsenal['year'].unique()
selected_year = st.selectbox(
    "What year would you like to observe?", 
    year_array)

pitch_type_array = arsenal['pitch_type_name'].unique()
pitch_type = st.selectbox(
    'Select Pitch:', 
    pitch_type_array)

In [14]:
#create sliders for pitch velocity and movement components
#use min/max on the data to limit the sliders

min_velo, max_velo=int(arsenal['avg_speed'].min()),int(arsenal['avg_speed'].max()+1)

velo_range = st.slider(
    "Choose Velocity Range", 
    value=(min_velo,max_velo),
    min_value=min_velo,max_value=max_velo)

min_vert, max_vert=arsenal['percent_rank_diff_z'].min(),arsenal['percent_rank_diff_z'].max()

vert_range = st.slider(
    "Choose Rise/Drop % vs. League Average", 
    value=(min_vert,max_vert),
    min_value=min_vert,max_value=max_vert)

min_break, max_break=arsenal['percent_rank_diff_x'].min(),arsenal['percent_rank_diff_x'].max()

break_range = st.slider(
    "Choose Break % vs. League Average",
    value=(min_break,max_break),
    min_value=min_break,max_value=max_break)

In [7]:
#filter the original data with the constraints applied from the sliders

filtered_pitchers=arsenal[(
    arsenal.year==selected_year) & (
    arsenal.pitch_type_name==pitch_type) & (
    arsenal.avg_speed<velo_range[1]) & (
    arsenal.avg_speed>velo_range[0]) & (
    arsenal.percent_rank_diff_z<vert_range[1]) & (
    arsenal.percent_rank_diff_z>vert_range[0]) & (
    arsenal.percent_rank_diff_x<break_range[1]) & (
    arsenal.percent_rank_diff_x>break_range[0])]

st.table(filtered_pitchers)

DeltaGenerator(_root_container=0, _provided_cursor=None, _parent=None, _block_type=None, _form_data=None)

In [24]:
#offer a scatter plot to chart the relationship between the components of a pitch and the results of the pitch

st.header('Outcome analysis')
st.write("""
###### Use the filtered data to plot an outcome statistic vs. velocity or a movement metric
z = vertical movement, x = horizontal movement
""")


fig1_x_axis=['avg_speed','percent_rank_diff_z','percent_rank_diff_x']
fig1_y_axis=['whiff_percent','run_value_per_100','woba']

fig1_x_choice = st.selectbox(
    'Input Metric: ', 
    fig1_x_axis)
fig1_y_choice = st.selectbox(
    'Output Metric: ', 
    fig1_y_axis)

fig1 = px.scatter(
    filtered_pitchers, 
    x=fig1_x_choice, 
    y=fig1_y_choice,
    hover_data=['name'])

fig1.update_layout(
title="<b> {} vs {} </b>".format(fig1_y_choice, fig1_x_choice))

st.plotly_chart(fig1)

DeltaGenerator(_root_container=0, _provided_cursor=None, _parent=None, _block_type=None, _form_data=None)

In [13]:
#offer an interactive histogram to let users see how the distribution of outcomes differs by pitch type

st.header('What pitches are getting the results?')
st.write("""
###### Use the full data set to map out a distribution of each pitch type vs. its outcome
""")


fig2_x_axis=fig1_y_axis

fig2_choice = st.selectbox(
    'Outcome Metric: ',
    fig2_x_axis)

fig2 = px.histogram(arsenal, x=fig2_choice, color='pitch_type')

fig2.update_layout(
title="<b> Distribituion of Pitch Types by {} </b>".format(fig2_choice))

st.plotly_chart(fig2)

DeltaGenerator(_root_container=0, _provided_cursor=None, _parent=None, _block_type=None, _form_data=None)